## Initial work
Modified the part that sets the folder paths, etc. (changed to the methods used in the PSR_MVPA_crossvalidation_fix.m file from the included .zip).

In [ ]:
%% Description              
%
% Per subject time-resolved cross-condition decoding of spatial frequency,
% where data is locked to S1 onset.
%
% Cross-condition comparisons are: 
%   fixation left  - fixation right
%   fixation right - fixation left
%   fixation left  - saccade
%   fixation right - saccade
%
% NB: train and test sets are balanced for spatial frequency, orientation 
%     and phase of the grating
%-------------------------------------------------------------------------%

%% Init                         
% initialize
clearvars; close all; clc;

% add paths to functions
PSR_setpaths;

% set folders
folderIN    = fullfile(cd,'TIMELOCKED','S1');
folderTRL   = fullfile(cd,'TRIALS','REJECTVISUAL');
folderOUT   = cd;

% subject
subjects = PSR_subjects(folderIN);

## Parameters
Below, the parameters are given, with their default values. This cell is tagged with the tagg "parameters". Papermill would inject a cell with parameters below this one, overriding the defaults with whatever setting you want.

In [ ]:
%% Parameters                   
bws        = -0.2; 	% baseline window start
bwe        = 0;   	% baseline window end
timeradius = 5;     % for temporal 'searchlight'
suffix     = {'LR','RL','LS','RS'};

## Core code
Left unchanged. Suggestion: consider throwing an error (or at least printing some message) if the run is aborted because the desired output file already exists.

In [ ]:
%% Loop over subjects           
for isubj = 1:numel(subjects)
    %% Print message            
    fprintf('\n%s\n',subjects{isubj});
    fileout = [folderOUT filesep subjects{isubj} '.mat'];
    if exist(fileout,'file')
        continue
    end
    
    %% Load data                
    channels = 'MEG';
    tic;
    [data]  = PSR_loadTimelock(folderIN,folderTRL,'S1',subjects{isubj},channels);
    dataTmp = data.s1lock;
    dataTmp = rmfield(dataTmp, 'cfg');
    dataTmp = ft_struct2double(dataTmp);
    toc;
    
    %% Preprocess               
    % 1. baseline correction
    cfg = [];
    cfg.demean = 'yes';
    cfg.baselinewindow = [bws bwe];
    [dataTmp] = ft_preprocessing(cfg,dataTmp);

    % 2. standardize data per sensors
    i1 = find( abs(dataTmp.time{1}-bws) == min(abs(dataTmp.time{1}-bws)) );
    i2 = find( abs(dataTmp.time{1}-bwe) == min(abs(dataTmp.time{1}-bwe)) );
    bw_std = std( cell2mat( cellfun(@(x) x(:,i1:i2),dataTmp.trial,'UniformOutput',false) ), [], 2);
    dataTmp.trial = cellfun(@(x) bsxfun(@rdivide,x,bw_std), dataTmp.trial,'UniformOutput',false);

    % 3. select conditions
    cfg         = [];
    cfg.trials  = PSR_selectCondition(dataTmp,{'fll','flh','frl','frh','srl','srh'});
    [dataTmp]   = ft_selectdata(cfg,dataTmp);

    % 4. make CoSMoMVPA data
    ds = PSR_mkCoSMoData(dataTmp);
    ds.sa.targets = ds.sa.sf;
    cosmo_check_dataset(ds);
    clear data dataTmp
    
    %% MVPA                     
    for s = 1:length(suffix)
        %% Partitions           
        % select data
        switch suffix{s}
            case 'LR'
                itrain = find(ds.sa.cndn==0 & ds.sa.fix == 0);
                itest  = find(ds.sa.cndn==0 & ds.sa.fix == 1);
            case 'RL'
                itrain = find(ds.sa.cndn==0 & ds.sa.fix == 1);
                itest  = find(ds.sa.cndn==0 & ds.sa.fix == 0);
            case 'LS'
                itrain = find(ds.sa.cndn==0 & ds.sa.fix == 0);
                itest  = find(ds.sa.cndn==1 & ds.sa.fix == 1);
            case 'RS'
                itrain = find(ds.sa.cndn==0 & ds.sa.fix == 1);
                itest  = find(ds.sa.cndn==1 & ds.sa.fix == 1);
        end
        
        % partitions
        partitions = PSR_mkCoSMoPartitions_BALANCE( ds, 1, itrain, 1, itest, 1 );
        partitions = cosmo_balance_partitions(partitions, ds);
        
        % time neigbourhood for searchlight in time
        time_nbrhood = cosmo_interval_neighborhood(ds,'time','radius',timeradius);

        %% SVM classification   
        % pre-allocation
        ntime       = length(ds.a.fdim.values{2});
        ntesttrials = size(partitions.test_indices{:},1);
        tmp_y       = int8( zeros( ntesttrials, ntime ) );
        tmp_yhat    = int8( zeros( ntesttrials, ntime ) );

        % print message
        fprintf('Classification %s, in progress... ', suffix{s} );
        tstart = tic;

        % slice along sample attribute dimension
        ds_train = cosmo_slice(ds, partitions.train_indices{:}, 1);
        ds_test  = cosmo_slice(ds, partitions.test_indices{:}, 1);
        
        % loop over time points
        parfor t = 1:ntime
            % time mask: current timepoint
            timemask    = time_nbrhood.neighbors{t};
            ds_trainTMP = cosmo_slice(ds_train, timemask, 2);
            ds_testTMP  = cosmo_slice(ds_test, timemask, 2);
            
            % SVM CLASSIFICATION
            pred = cosmo_classify_svm(ds_trainTMP.samples, ds_trainTMP.sa.targets, ds_testTMP.samples);
            
            % add to classification arrays - where time is aligned to S1
            tmp_y(:,t)    = ds_testTMP.sa.targets;
            tmp_yhat(:,t) = pred;
        end

        % print message
        fprintf('finished! Time elapsed: %.2f secs\n',toc(tstart));
        
        % Performance          
        % trials from all partitions
        S1.y.(suffix{s})    = double( reshape( permute(tmp_y,[1 3 2]), [], size(tmp_y,2), 1) );
        S1.yhat.(suffix{s}) = double( reshape( permute(tmp_yhat,[1 3 2]), [], size(tmp_yhat,2), 1) );
        S1.time = ds.a.fdim.values{2};
        
        % accuracy
        S1.accuracy.(suffix{s}) = mean( S1.y.(suffix{s}) == S1.yhat.(suffix{s}) );
        
        % sensitivity locked to S1 onset
        S1.dprime.(suffix{s}) = NaN(1,ntime);
        for t = 1:ntime
            S1.dprime.(suffix{s})(t) = computeDprime( S1.y.(suffix{s})(:,t)-1, S1.yhat.(suffix{s})(:,t)-1 );
        end
        
    end

    %% Save data                
    save( fileout, 'S1' );
        
    % remove large, redundant variables
    clear ds S1 ds_test ds_train time_nbrhood tmp_y tmp_yhat
    
end